In [ ]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import json
# Import the API key.
from config import weather_api_key
from config import gkey

In [ ]:
#Generate a list of 2000 random latitudes and Longitudes
lats = np.random.uniform(-90, 90, size=2000)
lngs = np.random.uniform(-90, 90, size=2000)
coordinates = zip(lats, lngs)

In [ ]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [ ]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    try:
        # to Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather['weather'][0]['description']
        city_country = city_weather['sys']['country']
        

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                        "Country": city_country,
                        "Lat": city_lat,
                        "Lng": city_lng,
                        "Max Temp": city_max_temp,
                        "Humidity": city_humidity,
                        "Cloudiness": city_clouds,
                        "Wind Speed": city_wind,
                        "Description": city_description})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.rename(columns={'Description' : 'Current Description'}, inplace = True)

# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

city_data_df.head()